# Prelab #2

In [5]:
import jlab as jl
from importlib import reload
reload(jl)

<module 'jlab' from '/Users/n1le/Documents/Classes/PHYS 3330/jlab.py'>

## Section 2.2, Voltage Dividers
2.2.1 What is the current I through each resistor in a voltage divider?

$I = V/R = V_{in} / (R_1 + R_2) $

2.2.2 What is the voltage across R_2? Express WRT current.

The current is constant in a voltage divider, so $I = V_{in} / (R_1 + R_2)$ and we can then calculate the voltage from Ohm's Law: $V_{out} = IR = V_{in} R_2 / (R_1 + R_2)$. This is $V_{out}$ since it's the voltage coming out of the top resistor with respect to ground (0V).

2.2.3 As above, $V_{out} = V_{in}R_2 / (R_1 + R_2)$. The following code calculates the output voltages at each node of a voltage divider. Parameters:
- v_in (float): The input voltage in volts.
- resistors (list): A list of resistance values in ohms.

Returns:
- list: A list of output voltages at each node in volts.
```
def voltage_divider(v_in: float, resistors: list) -> list:
    total_resistance = sum(resistors)
    voltages = []
    v_out = v_in
    for r in resistors:
        v_drop = v_out * (r / total_resistance)
        voltages.append(v_drop)
        v_out -= v_drop
        total_resistance -= r
    return voltages
```
2.2.4 Build the circuit ![Circuit Diagram](Images/ideal-vd.png) in LTSpice with
- $V_{in} = 10V$
- $R_1 = 1 k\Omega$
- $R_2 = 3 k\Omega$.
Calculate $I$ and $V_{out}$ using results from the previous questions, then run a simulation to measure the two.

Numerically: $V_{out} = V_{in}R_2 / (R_1 + R_2) = 7.5 V$, $I = V_{in} / (R_1 + R_2) = 10/ 4 \cdot 10^{3} = 2.5 \cdot 10^{-2} A = 2.5 mA$

In [6]:
vin = 10.0
r1 = 1 * 10**3  # 1 kOhm
r2 = 3 * 10**3  # 3 kOhm
jl.voltage_divider(vin, [r1, r2])

[7.5]

Code, numerical solutions, and simulation all agree!

![LTSpice Screenshots](Images/PLab2-2.2.4.1.png)
![LTSpice Screenshots](Images/PLab2-2.2.4.2.png)

## Section 2.3, Transfer Function
2.3.1 Write down the equatino for the transfer function of the ideal voltage divider.

$ T = \frac{V_{out}}{V_{in}} = \frac{V_{in} R_2 / (R_1 + R_2) }{V_in} = \frac{R_2}{ (R_1 + R_2)}$

2.3.2 For $R_1 = 2k\Omega$ and $R_2 = 1 k\Omega$, what is the value of the transfer function?

Simply plugging in values: $T = \frac{1k\Omega}{3k\Omega} = \frac 1 3$

In [7]:
r1 = 2 * 10**3  # 2 kOhm
r2 = 1 * 10**3  # 1 kOhm
jl.transfer_function_voltage_divider([r1, r2])

[0.3333333333333333]

Calculated and code match!

2.3.3 For a $V_{in} = 10V$, what will $V_{out}$ be? In LTSpice, simulate the output voltage.

$V_{out} = T \cdot V_{in} = \frac 1 3 \cdot 10V = 10/3 V \approx 3.33 V$

![Circuit Simulation](Images/PLab2-2.3.3.1.png)
![Circuit Simulation](Images/PLab2-2.3.3.2.png)


## 2.4 Input and Output Impedance
![Real Power Source](Images/output-input-impedance.png)

2.4.1 Confirm that the above transfer function from 2.3.1 is consistent with the transfer function $T = \frac{V^{(ext)}_{supply}}{V^{(int)}_{supply}} = \frac{R_i}{R_o + R_i}$

Treating the $R_o$ and $R_i$ resistors as being the voltage divider, we do indeed get $T = \frac{R_i}{R_0 + R_i}$.

2.4.2 The power delivered is $P = I^2 R_i = IV_{supply}^{(ext)} = \frac{(V_{supply}^{(ext)})^2}{R_i}$. For a given $R_o$, find the $R_i$ that maximizes the power delivered.

$$\frac{\partial P}{\partial R_i} = \frac{\partial}{\partial R_i} (V_{supply}^{(ext)})^2 / R_i = \frac{\partial}{\partial R_i} (TV_{supply}^{(int)})^2 / R_i $$
$$ = \frac{\partial}{\partial R_i} \frac{ \left( \frac{R_i}{R_o + R_i} V_{supply}^{(int)} \right)^2 }{R_i} = - \frac{(R_i - R_o)(V_{supply}^{(int)})^2 }{(R_i + R_o)^3} = 0$$
$$ \implies R_o = R_i$$

2.4.4 Plot $P$ vs $R_i$ from $R_i = 0 \Omega$ to $R_i = 100 \Omega$ using $V_{supply}^{(int)} = 1V$ with the following values of $R_o$ on the same plot
- $R_o = 10 \Omega$
- $R_o = 25 \Omega$
- $R_o = 50 \Omega$

In [8]:
def power(r_o, r_i, v):
    r_i = np.asarray(r_i)
    return np.where(r_i == 0, 0.0, v**2 * r_i / (r_o + r_i)**2)

volt = 1
Ri = np.linspace(0, 100, 1000)  # input impedance
output_impedances = (10, 25, 50)

fig, ax = plt.subplots(1, 1, figsize=(4, 3))
for Ro in output_impedances:
    ax.plot(Ri, power(Ro, Ri, volt), label=f'{Ro} $\\Omega$')
ax.set_xlabel('Input Impedance ($\\Omega$)')
ax.set_ylabel('Power (W)')
ax.set_title('Power delivered from non-ideal voltage source')
ax.set_xticks(np.arange(0, 101, 10))
ax.set_xticks(np.arange(0, 101, 2), minor=True)
ax.set_xlim(0, 100)
ax.set_ylim(0, .026)
ax.minorticks_on()
ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
ax.legend(title="Output Impedance")
ax.grid(linestyle="dotted")
fig.tight_layout()
plt.show()
#fig.savefig('impedance_matching_plot.png', dpi=600, bbox_inches='tight')

NameError: name 'np' is not defined

## 2.5 Voltage Divider with Non-ideal Power Supply
![Real Power Supply With Voltage Divider](Images/nonideal-vd.png)

2.5.1 What is the input impedance of the voltage divider circuit? 

This is just $R_1 + R_2$

2.5.2 The $V_{in}$ of the voltage divider will be the $V_{supply}^{(ext)} from the power supply. Express $V_{in}$ as a function of $V_{supply}^{(int)}$ and the resistor values.

$V_{in} = V_{supply}^{(int)} \frac{R_1 + R_2}{R_o + R_1 + R_2}$

2.5.3 Using the voltage divider, express $V_{out}$ of the circuit with respect to $V_{supply}^{(int)}$ and the resistor value. Write a python function that computes the output voltage of a voltage divider with a non-ideal voltage supply.

$V_{out} = V_{supply}^{(int)} \frac{R_2}{R_o + R_1 + R_2}$



In [ ]:
def nonideal_voltage_divider_vout(V_int: float, R_o: float, R1: float, R2: float) -> float:
    """
    Compute V_out for a voltage divider fed by a non-ideal supply.
    V_out = V_int * R2 / (R_o + R1 + R2)

    Args:
        V_int: internal supply voltage (V_supp^{(int)})
        R_o: source output impedance
        R1: top resistor of divider
        R2: bottom resistor of divider

    Returns:
        V_out (float or numpy array)
    """
    denom = R_o + R1 + R2
    try:
        return V_int * R2 / denom
    except ZeroDivisionError:
        return 0.0

2.5.4 For an ideal voltage divider ($R_o = 0 \Omega$) having $R_1 = 100 \Omega$ and $R_2 = 50 \Omega$ or $R_1 = 2k \Omega$ and $R_2 = 1 k\Omega$ or $R_1 = 200 k\Omega$ and $R_2 = 100 k\Omega$ will have the same transfer function. Predict $V_{out}$ when the ideal voltage source is set to $10V$. Build these circuits in LTSpice.

$V_{out} = 10 V \cdot \frac{R_2}{R_1 + R_2}$
- $R_1 = 100 \Omega$ and $R_2 = 50 \Omega$ $V_{out} = 10V \cdot \frac{50 \Omega}{150 \Omega} \approx 3.33V$
- $R_1 = 2 k\Omega$ and $R_2 = 1 k\Omega$ $V_{out} = 10V \cdot \frac{1k\Omega}{3k\Omega} = 3.33V$
- $R_1 = 200 k\Omega$ and $R_2 = 100 k\Omega$ $V_{out} = 10V \cdot \frac{100k\Omega}{300k\Omega} = 3.33V$

![Circuit Simulation](Images/PLab2-2.5.4.1.png)
![Circuit Simulation](Images/PLab2-2.5.4.2.png)

All the values match exactly.

2.5.5 What is $V_{out}$ for each 2.5.4, but with an output impedance of $R_o = 50 \Omega$?

- $R_1 = 100 \Omega$ and $R_2 = 50 \Omega$ $V_{out} = 10V \cdot \frac{50 \Omega}{200 \Omega} \approx 2.5V$
- $R_1 = 2 k\Omega$ and $R_2 = 1 k\Omega$ $V_{out} = 10V \cdot \frac{1k\Omega}{3050\Omega} = 3.27V$
- $R_1 = 200 k\Omega$ and $R_2 = 100 k\Omega$ $V_{out} = 10V \cdot \frac{100k\Omega}{300,050\Omega} = 3.33V$

Circuit is the same with $50 \Omega$ impedance on the voltage source. Simulated values match calculation exactly.

![Circuit Simulation](Images/PLab2-2.5.5.1.png)

2.5.6 
- Compare the output voltage of the voltage divider with an ideal voltage source vs a non-ideal voltage source.
    - When the voltage source is ideal, it provides a much more stable voltage than if it is non-ideal. Non-ideal voltage sources mess with the voltage of the output circuit and must be accounted for if you need an accurate measurement.
- How does the input impedance of the voltage divider $(R_1 + R_2)$ impact the non-ideal circuit compared to the ideal circuit?
    - If the input impedance is much much larger than the resistance of the voltage source, you get behavior that approximates an ideal voltage source. However, when the input impedance near or on the same order of magnitude, the behavior becomes much less stable.
- What condition should be met such that both the ideal and non-ideal voltage source models sufficiently agree?
    - In order to keep the ideal and non-ideal models in sufficient agreement, we require $R_o << (R_1 + R_2)$.

## Impacts of Parallel Impedances
![Circuit Diagram](https://ucboulder.github.io/PHYS-3330/resources/lab2fig/modified-vd.png)

2.6.1 Rearrange $T = \frac{R_2 R_3}{R_1R_2 + R_1R_3 + R_2R_3}$ for $R_3$.

$$ TR_1R_2 + TR_1R_3 + TR_2R_3 = R_2R_3 $$
$$ TR_1R_2 = (R_2 - TR_1 - TR_2)R_3 $$
$$ R_3 = \frac{TR_1R_2}{R_2 - TR_1 - TR_2}$$
$$ R_3 = \frac{TR_1R_2}{R_2 -T(R_1 + R_2)}$$



In [ ]:
def R_3(V_in, V_out, R_1, R_2):
    """
    Compute the required load resistance R_3 to achieve a desired V_out
    from a voltage divider with input voltage V_in and resistors R_1 and R_2.

    Args:
        V_in: input voltage
        V_out: desired output voltage
        R_1: top resistor of divider
        R_2: bottom resistor of divider
    Returns:
        R_3 = TR_1R_2 / (R_2 - T(R_1 - R_2)) where T = V_out / V_in
    """
    T = V_out / V_in
    numerator = T * R_1 * R_2
    denominator = R_2 - T * (R_1 - R_2)
    try:
        return numerator / denominator
    except ZeroDivisionError:
        return np.inf
    
print( R_3(15, 7.5, 0.934e3, 0.984e3) )
print( R_3(15, 7.1, 1e6, 0.992e6) )
print( R_3(15, 5.16, 10.87e6, 10.27e6))

455.4291377601586
475147.06676021364
3815970.9845383368


In [13]:
def modified_voltage_divider(V_in, R_1, R_2, R_3):
    """
    Compute V_out for a voltage divider with load resistance R_3.

    Args:
        V_in: input voltage
        R_1: top resistor of divider
        R_2: bottom resistor of divider
        R_3: load resistance
    Returns:
        V_out = V_in * (R_4 / (R_1 + R_4)) where R_4 = (R_2 * R_3) / (R_2 + R_3)
    """
    R_4 = (R_2 * R_3) / (R_2 + R_3)
    return V_in * (R_4 / (R_1 + R_4))

print("V_in = 15V, R_1 = 0.934 kOhm, R_2 = 0.984 kOhm, R_3 = 10MOhm, V_out =",
      modified_voltage_divider(15, 0.934e3, 0.984e3, 5e6), 
      "Percent error: ", jl.percent_error(modified_voltage_divider(15, 0.934e3, 0.984e3, 5e6), 7.5))
print("V_in = 15V, R_1 = 1.000MOhm, R_2 = 0.992 MOhm, R_3 = 10MOhm, V_out =",
      modified_voltage_divider(15, 1e6, 0.992e6, 5e6), 
      "Percent error: ", jl.percent_error(modified_voltage_divider(15, 1e6, 0.992e6, 5e6), 7.1))
print("V_in = 15V, R_1 = 10.87MOhm, R_2 = 10.27 MOhm, R_3 = 10MOhm, V_out =",
      modified_voltage_divider(15, 10.87e6, 10.27e6, 10e6),
      "Percent error: ", jl.percent_error(modified_voltage_divider(15, 10.87e6, 10.27e6, 10e6), 5.16))

V_in = 15V, R_1 = 0.934 kOhm, R_2 = 0.984 kOhm, R_3 = 10MOhm, V_out = 7.694778734873309 Percent error:  2.5970497983107896
V_in = 15V, R_1 = 1.000MOhm, R_2 = 0.992 MOhm, R_3 = 10MOhm, V_out = 6.793279766252739 Percent error:  4.320003292214944
V_in = 15V, R_1 = 10.87MOhm, R_2 = 10.27 MOhm, R_3 = 10MOhm, V_out = 4.7688345748400565 Percent error:  7.580725293797358


## 2.7 Thevenin's Theorem and the Voltage Divider
![Thevenin Equivalent Circuit](https://ucboulder.github.io/PHYS-3330/resources/lab2fig/thevenin-vd.png)

$$V_T = V_{in} \frac{R_2}{R_1 + R_2}$$
$$R_o = \frac{R_1R_2}{R_1 + R_2}$$

2.7.1 Use the result of 2.6.1 to show that the voltage across $R_3$ is the same as the voltage predicted by the Thevenin equivalent circuit.

We have $V_{R_3} = V_T \cdot \frac{R_3}{R_o + R_3}$, so plugging in the definitions above, and 2.6.1
$$ V_{R_3} = \left( V_{in} \frac{R_2}{R_1 + R_2} \right)^2 \cdot \frac{R_3}{\frac{R_1R_2}{R_1+R_2} + R_3} $$
then multiply both top and bottom by $R_1 + R_2$
$$ V_{R_3} = V_{in} \cdot \frac{R_2R_3}{R_1R_2 + R_3(R_1 + R_2)}.$$
Now the transmission coefficient for this problem is $T = \frac{V_{R_3}}{V_{in}}$, so from above
$$T = \frac{R_2 R_3}{R_1R_2 + R_3(R_1+R_2)}$$
which, after some algebra, gives
$$ R_2 R_3 = TR_1R_2 + TR_3 (R_1 + R_2)$$
$$ R_2 R_3 - TR_3 (R_1 + R_2) = TR_1R_2$$
$$ R_3 (R_2 - T(R_1+R_2)) = TR_1 R_2$$
$$ R_3 = \frac{TR_1 R_2}{R_2 - T(R_1 + R_2)}$$
which is exactly what was computed above.